## This notebook contains the code implemented in the first task of my virtual internship with KPMG through Forage.

For this task I was instructed to take a look at the four datasets provided by Sprocket Central Pty Ltd and identify all data quality issues.

In [1]:
#importing relevant libaries
import numpy as np
import pandas as pd
import datetime
import sqlite3 as sql
from pandas import ExcelWriter

In [2]:
#reading in the needed sheets from the excel workbook
KPMG_TSC = pd.read_excel('c:Documents/KPMG data.xlsx', sheet_name = 'Transactions')
KPMG_NCL = pd.read_excel('c:Documents/KPMG data.xlsx', sheet_name = 'NewCustomerList')
KPMG_CD = pd.read_excel('c:Documents/KPMG data.xlsx', sheet_name = 'CustomerDemographic')
KPMG_CA = pd.read_excel('c:Documents/KPMG data.xlsx', sheet_name = 'CustomerAddress')

In [3]:
#changing the maximum number of rows and columns that can the viewed
pd.options.display.max_rows = 200
pd.options.display.max_columns = 200

# Checking the quality of the Transactions dataset

In [4]:
#viewing the transaction dataset
KPMG_TSC.head()

,Note: The data and information in this document is reflective of a hypothetical situation and client. This document is to be used for KPMG Virtual Internship purposes only.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
1,1,2,2950,2017-02-25 00:00:00,False,Approved,Solex,Standard,medium,medium,71.49,53.62,41245
2,2,3,3120,2017-05-21 00:00:00,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701
3,3,37,402,2017-10-16 00:00:00,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361
4,4,88,3135,2017-08-31 00:00:00,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.1,36145


In [5]:
#cleaning all datasets to replace the column head with the first row
datasets = [KPMG_TSC, KPMG_NCL, KPMG_CD, KPMG_CA]
for data in datasets:
    data.columns = data.iloc[0]
    data.drop(0, axis = 'index', inplace = True)

In [6]:
#checking the details of the transaction dataset
KPMG_TSC.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 1 to 20000
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   transaction_id           20000 non-null  object
 1   product_id               20000 non-null  object
 2   customer_id              20000 non-null  object
 3   transaction_date         20000 non-null  object
 4   online_order             19640 non-null  object
 5   order_status             20000 non-null  object
 6   brand                    19803 non-null  object
 7   product_line             19803 non-null  object
 8   product_class            19803 non-null  object
 9   product_size             19803 non-null  object
 10  list_price               20000 non-null  object
 11  standard_cost            19803 non-null  object
 12  product_first_sold_date  19803 non-null  object
dtypes: object(13)
memory usage: 1015.7+ KB


The following columns all have missing values: 
1. online_order
2. brand
3. product_line
4. product_class
5. product_size 
6. standard_cost
7. product_first_sold_date 

In [7]:
#checking the number of missing values each column has
KPMG_TSC.isnull().sum()

0
transaction_id               0
product_id                   0
customer_id                  0
transaction_date             0
online_order               360
order_status                 0
brand                      197
product_line               197
product_class              197
product_size               197
list_price                   0
standard_cost              197
product_first_sold_date    197
dtype: int64

In [8]:
#checking for correlation in the position of missing values across the last 6 columns with missing values
KPMG_TSC[KPMG_TSC['brand'].isnull()]

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
137,137,0,431,2017-09-23 00:00:00,False,Approved,NaN,NaN,NaN,NaN,1942.61,NaN,NaN
160,160,0,3300,2017-08-27 00:00:00,False,Approved,NaN,NaN,NaN,NaN,1656.86,NaN,NaN
367,367,0,1614,2017-03-10 00:00:00,False,Approved,NaN,NaN,NaN,NaN,850.89,NaN,NaN
407,407,0,2559,2017-06-14 00:00:00,True,Approved,NaN,NaN,NaN,NaN,710.59,NaN,NaN
677,677,0,2609,2017-07-02 00:00:00,False,Approved,NaN,NaN,NaN,NaN,1972.01,NaN,NaN
781,781,0,897,2017-05-10 00:00:00,True,Approved,NaN,NaN,NaN,NaN,311.54,NaN,NaN
1004,1004,0,2254,2017-05-16 00:00:00,False,Cancelled,NaN,NaN,NaN,NaN,205.84,NaN,NaN
1131,1131,0,2163,2017-03-09 00:00:00,False,Approved,NaN,NaN,NaN,NaN,605.54,NaN,NaN
1197,1197,0,2234,2017-04-04 00:00:00,False,Approved,NaN,NaN,NaN,NaN,756.31,NaN,NaN
1283,1283,0,397,2017-08-09 00:00:00,False,Approved,NaN,NaN,NaN,NaN,850,NaN,NaN


The columns :brand, product_line, product_class, product_size, standard_cost, product_first_sold_date have all of their missing values in the same rows while 358 of the remaining missing values are in the 'online_order' column. Therefore the total number of null values in the transaction dataset is the sum of the different groups of missing values.

Although other columns with missing values may be filled with the mode values, the last column 'product_first_sold_date' is unique to each product, thus all missing vales in the dataset are dropped.

In [9]:
#calculating the total number of missing values
percentage_onlineorder = ((((KPMG_TSC['online_order'].isnull().sum()) - 2) * 100) / (len((KPMG_TSC['online_order']))))
percentage_brand = ((KPMG_TSC['brand'].isnull().sum() * 100) / (len((KPMG_TSC['brand']))))
percentage_nan = (percentage_onlineorder + percentage_brand)
percentage_nan

2.775

In [10]:
#dropping the null values in the datatset
KPMG_TSC.dropna(inplace = True)
KPMG_TSC.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19445 entries, 1 to 20000
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   transaction_id           19445 non-null  object
 1   product_id               19445 non-null  object
 2   customer_id              19445 non-null  object
 3   transaction_date         19445 non-null  object
 4   online_order             19445 non-null  object
 5   order_status             19445 non-null  object
 6   brand                    19445 non-null  object
 7   product_line             19445 non-null  object
 8   product_class            19445 non-null  object
 9   product_size             19445 non-null  object
 10  list_price               19445 non-null  object
 11  standard_cost            19445 non-null  object
 12  product_first_sold_date  19445 non-null  object
dtypes: object(13)
memory usage: 1.1+ MB


In [11]:
#recasting some of the columns in appropriate data types
KPMG_TSC['transaction_date'] = pd.to_datetime(KPMG_TSC['transaction_date'])
KPMG_TSC['list_price'] = KPMG_TSC['list_price'].astype('float')
KPMG_TSC['standard_cost'] = KPMG_TSC['standard_cost'].astype('float')
KPMG_TSC['product_first_sold_date'] = KPMG_TSC['product_first_sold_date'].apply(lambda x: (datetime.datetime(1899, 12, 30) + datetime.timedelta(days= int(x))))
KPMG_TSC.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19445 entries, 1 to 20000
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   transaction_id           19445 non-null  object        
 1   product_id               19445 non-null  object        
 2   customer_id              19445 non-null  object        
 3   transaction_date         19445 non-null  datetime64[ns]
 4   online_order             19445 non-null  object        
 5   order_status             19445 non-null  object        
 6   brand                    19445 non-null  object        
 7   product_line             19445 non-null  object        
 8   product_class            19445 non-null  object        
 9   product_size             19445 non-null  object        
 10  list_price               19445 non-null  float64       
 11  standard_cost            19445 non-null  float64       
 12  product_first_sold_date  19445 n

In [12]:
#checking that all changes have been accurately implemented
KPMG_TSC.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
1,1,2,2950,2017-02-25,False,Approved,Solex,Standard,medium,medium,71.49,53.62,2012-12-02
2,2,3,3120,2017-05-21,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,2014-03-03
3,3,37,402,2017-10-16,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,1999-07-20
4,4,88,3135,2017-08-31,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,1998-12-16
5,5,78,787,2017-10-01,True,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,2015-08-10


In [13]:
#checking for duplicates in the dataset
len(KPMG_TSC[KPMG_TSC.duplicated()])

0

There were no duplicates found in the dataset

## checking each column for data acuracy

In [14]:
#checking for accuracy in the product_id column
KPMG_TSC.product_id.value_counts().head()

0     1157
3      349
1      305
38     267
35     261
Name: product_id, dtype: int64

In [15]:
KPMG_TSC[KPMG_TSC.product_id == 0]['brand'].value_counts()

Solex             273
OHM Cycles        240
Norco Bicycles    238
Trek Bicycles     212
Giant Bicycles    101
WeareA2B           93
Name: brand, dtype: int64

In [16]:
KPMG_TSC[KPMG_TSC.product_id == 3]['brand'].value_counts()

Trek Bicycles    349
Name: brand, dtype: int64

In [17]:
KPMG_TSC[KPMG_TSC.product_id == 1]['brand'].value_counts()

Giant Bicycles    305
Name: brand, dtype: int64

In [18]:
KPMG_TSC[KPMG_TSC.product_id == 38]['brand'].value_counts()

Solex            156
Trek Bicycles    111
Name: brand, dtype: int64

In [19]:
KPMG_TSC[KPMG_TSC.product_id == 35]['brand'].value_counts()

Trek Bicycles     138
Giant Bicycles    123
Name: brand, dtype: int64

Some product id belong to only one brand while others belong to multiple brands. This raises concerns about the accuracy of the product ids


In [20]:
#checking for accuracy in the customer_id column
KPMG_TSC.customer_id.value_counts().head()

1068    14
2476    14
2183    14
1302    13
2912    13
Name: customer_id, dtype: int64

In [21]:
#checking for accuracy in the transaction_date column
KPMG_TSC.transaction_date.value_counts().head()

2017-08-18    81
2017-02-14    81
2017-10-15    73
2017-01-31    72
2017-12-19    70
Name: transaction_date, dtype: int64

In [22]:
#checking for accuracy in the online_order column
KPMG_TSC.online_order.value_counts()

True     9739
False    9706
Name: online_order, dtype: int64

In [23]:
#checking for accuracy in the order_status column
KPMG_TSC.order_status.value_counts()

Approved     19273
Cancelled      172
Name: order_status, dtype: int64

In [24]:
#checking for accuracy in the brand column
KPMG_TSC.brand.value_counts()

Solex             4169
WeareA2B          3245
Giant Bicycles    3244
OHM Cycles        2993
Trek Bicycles     2931
Norco Bicycles    2863
Name: brand, dtype: int64

In [25]:
#checking for accuracy in the product_line column
KPMG_TSC.product_line.value_counts()

Standard    13920
Road         3894
Touring      1213
Mountain      418
Name: product_line, dtype: int64

In [26]:
#checking for accuracy in the product_class column
KPMG_TSC.product_class.value_counts()

medium    13587
high       2952
low        2906
Name: product_class, dtype: int64

In [27]:
#checking for accuracy in the product_size column
KPMG_TSC.product_size.value_counts()

medium    12767
large      3900
small      2778
Name: product_size, dtype: int64

In [28]:
#checking for accuracy in the product_first_sold_date column
KPMG_TSC.product_first_sold_date.value_counts().head()

1992-10-02    230
2012-06-04    223
2003-07-21    221
2009-03-08    220
2004-08-17    217
Name: product_first_sold_date, dtype: int64

Major data quality issues found in the transactions dataset:

1.  Missing values:  The dataset had 360 (1.8%) missing values in the 'online_order' column . The columns 'brand', 'product_line', 'product_class', 'product_size', 'standard_cost', 'product_first_sold_date' all had 197 (0.985% ) missing values spanning across the same rows.

2.  Wrong data type of columns: Some columns had values presented in wrong data types. e.g  'product_first_sold_date'.

3.  Column accuracy: In the product_id column, some product ids belong to only one brand while others belong to multiple brands. This may lead to the assumation that no brand of product has a unique product id.

Mitigation:  A total of 2.8% missing values were removed from the dataset and appropriate data transformations were made to ensure consistent data types for a given field.

Recommendation:  Ensure that fact tables in the given database have constraints on data types. 

# Checking for data quality in the NewCustomerList dataset

In [29]:
#Viewing the dataset
KPMG_NCL.head()

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,NaN,NaN,NaN,NaN,NaN,Rank,Value
1,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,N,Yes,14,45 Shopko Center,4500,QLD,Australia,6,0.63,0.7875,0.984375,0.836719,1.0,1,1.71875
2,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,16,14 Mccormick Park,2113,NSW,Australia,11,0.81,0.8100,1.012500,0.860625,1.0,1,1.71875
3,Ardelis,Forrester,Female,10,1974-08-28 00:00:00,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,10,5 Colorado Crossing,3505,VIC,Australia,5,0.69,0.6900,0.690000,0.690000,1.0,1,1.71875
4,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,N,Yes,5,207 Annamark Plaza,4814,QLD,Australia,1,0.44,0.5500,0.550000,0.550000,4.0,4,1.703125
5,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,N,No,19,115 Montana Place,2093,NSW,Australia,9,0.54,0.5400,0.675000,0.675000,4.0,4,1.703125


In [30]:
#checking the details of the column
KPMG_NCL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 1 to 1000
Data columns (total 23 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   first_name                           1000 non-null   object 
 1   last_name                            971 non-null    object 
 2   gender                               1000 non-null   object 
 3   past_3_years_bike_related_purchases  1000 non-null   object 
 4   DOB                                  983 non-null    object 
 5   job_title                            894 non-null    object 
 6   job_industry_category                835 non-null    object 
 7   wealth_segment                       1000 non-null   object 
 8   deceased_indicator                   1000 non-null   object 
 9   owns_car                             1000 non-null   object 
 10  tenure                               1000 non-null   object 
 11  address                       

In [31]:
#number of missing values in each column
KPMG_NCL.isnull().sum()

0
first_name                               0
last_name                               29
gender                                   0
past_3_years_bike_related_purchases      0
DOB                                     17
job_title                              106
job_industry_category                  165
wealth_segment                           0
deceased_indicator                       0
owns_car                                 0
tenure                                   0
address                                  0
postcode                                 0
state                                    0
country                                  0
property_valuation                       0
NaN                                      0
NaN                                      0
NaN                                      0
NaN                                      0
NaN                                      0
Rank                                     0
Value                                    0
dtype: in

The columns: job_title and job_industry_category have the highest number of missing values however these columns are not sensitive enough to require being dropped from dataset but cannot be replaced by the mode of the column due to the need for accuracy hence each null value will be replaced with "N/A" (Not Available)

In [32]:
#filling the null values in both columns
KPMG_NCL['job_title'].fillna('N/A', inplace = True)
KPMG_NCL['job_industry_category'].fillna('N/A', inplace = True)
KPMG_NCL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 1 to 1000
Data columns (total 23 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   first_name                           1000 non-null   object 
 1   last_name                            971 non-null    object 
 2   gender                               1000 non-null   object 
 3   past_3_years_bike_related_purchases  1000 non-null   object 
 4   DOB                                  983 non-null    object 
 5   job_title                            1000 non-null   object 
 6   job_industry_category                1000 non-null   object 
 7   wealth_segment                       1000 non-null   object 
 8   deceased_indicator                   1000 non-null   object 
 9   owns_car                             1000 non-null   object 
 10  tenure                               1000 non-null   object 
 11  address                       

In [33]:
#checking for the positions of null values in last name column to see if it coincides with the null values in DOB column
KPMG_NCL[KPMG_NCL['last_name'].isnull()]

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,NaN,NaN,NaN,NaN,NaN,Rank,Value
13,Olag,NaN,Male,60,1990-05-13,Human Resources Manager,Telecommunications,Mass Customer,N,No,9,0484 North Avenue,2032,NSW,Australia,11,0.93,0.9300,1.162500,0.988125,13.0,13,1.609375
59,Whittaker,NaN,Male,64,1966-07-29,Media Manager III,N/A,Mass Customer,N,Yes,8,683 Florence Way,3156,VIC,Australia,5,0.73,0.9125,0.912500,0.775625,57.0,57,1.375
88,Kahaleel,NaN,Male,5,1942-11-01,GIS Technical Architect,N/A,High Net Worth,N,No,13,12 Arapahoe Park,2035,NSW,Australia,12,0.66,0.6600,0.825000,0.825000,88.0,88,1.314844
156,Bill,NaN,Female,74,1963-04-24,Human Resources Assistant II,Property,Mass Customer,N,Yes,19,6704 Pine View Lane,2170,NSW,Australia,9,0.67,0.8375,1.046875,0.889844,155.0,155,1.2
203,Glyn,NaN,Male,47,1945-02-13,General Manager,Manufacturing,Affluent Customer,N,Yes,21,67 Bluejay Plaza,2300,NSW,Australia,9,0.58,0.7250,0.906250,0.906250,202.0,202,1.140625
327,Haleigh,NaN,Female,17,1952-05-19,Senior Sales Associate,Financial Services,Mass Customer,N,Yes,18,49 Jana Point,4503,QLD,Australia,4,0.97,1.2125,1.212500,1.030625,326.0,326,1.009375
331,Alon,NaN,Male,17,1999-06-23,Accountant IV,N/A,Affluent Customer,N,No,9,770 Crest Line Parkway,4218,QLD,Australia,3,0.67,0.6700,0.670000,0.670000,329.0,329,1
358,Otis,NaN,Male,59,1971-01-11,Electrical Engineer,Manufacturing,Affluent Customer,N,No,12,04 Oakridge Plaza,2075,NSW,Australia,11,0.42,0.4200,0.525000,0.525000,358.0,358,0.98
420,Sherill,NaN,Female,33,1991-12-18,Information Systems Manager,Financial Services,Mass Customer,N,No,3,53 Moulton Avenue,2880,NSW,Australia,1,1.04,1.0400,1.040000,0.884000,420.0,420,0.91375
443,Theresina,NaN,Female,30,1987-03-01,General Manager,Argiculture,Mass Customer,N,Yes,14,253 Katie Junction,2650,NSW,Australia,2,0.75,0.9375,0.937500,0.796875,441.0,441,0.901


In [34]:
#Recasting in the appropriate datatypes
KPMG_NCL['past_3_years_bike_related_purchases'] = KPMG_NCL['past_3_years_bike_related_purchases'].astype('float')
KPMG_NCL['DOB'] = pd.to_datetime(KPMG_NCL['DOB'])
KPMG_NCL['tenure'] = KPMG_NCL['tenure'].astype('int')
KPMG_NCL['property_valuation'] = KPMG_NCL['property_valuation'].astype('int')
KPMG_NCL['Rank'] = KPMG_NCL['Rank'].astype('int')
KPMG_NCL['Value'] = KPMG_NCL['Value'].astype('float')
KPMG_NCL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 1 to 1000
Data columns (total 23 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   first_name                           1000 non-null   object        
 1   last_name                            971 non-null    object        
 2   gender                               1000 non-null   object        
 3   past_3_years_bike_related_purchases  1000 non-null   float64       
 4   DOB                                  983 non-null    datetime64[ns]
 5   job_title                            1000 non-null   object        
 6   job_industry_category                1000 non-null   object        
 7   wealth_segment                       1000 non-null   object        
 8   deceased_indicator                   1000 non-null   object        
 9   owns_car                             1000 non-null   object        
 10  tenure      

In [35]:
#Checking for duplicates
len(KPMG_NCL[KPMG_NCL.duplicated()])

0

There were no duplicates found in the dataset

## Checking for accuracy in each column

In [36]:
#checking for accuracy in the gender column
KPMG_NCL['gender'].value_counts()

Female    513
Male      470
U          17
Name: gender, dtype: int64

The gender column has another gender stated as 'U'. It is unclear what this stands for and thus raises concerns about the accuarcy of the column.

In [37]:
#checking for accuracy in the wealth_segment column
KPMG_NCL['wealth_segment'].value_counts()

Mass Customer        508
High Net Worth       251
Affluent Customer    241
Name: wealth_segment, dtype: int64

In [38]:
#removing the missing values from the dataset
KPMG_NCL.dropna(inplace = True)

Major data quality issues in the New Customer List dataset:

1. Missing Values: The dataset had 106  (10.6%) missing values in job_title and 165 (16.5%) missing values in job_industry_category columns. It also had 29 customers (2.9%) whose last names were not documented and another 17 (1.7%) had missing date of births.

2. Column Accuracy: The gender column had accuracy issues. It had another category 'U' other than 'female' and 'male'.

3. Missing columns titles: Five columns, positioned between property_valuation and rank had no column title.

Mitigation:  Replaced the missing values in job_title and  job_industry_category  with 'N/A' (Not Available) to reduce the number of missing values from 285 to 46.

Recommendation: Enforce a drop-down list for the user entering the data rather than a free text field to increase column accuracy. Review all data carefully before separating into customer demographic and customer address datasets.


# Checking for data quality in the Customer Demographic dataset

In [39]:
#viewing the dataset
KPMG_CD.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
1,1,Laraine,Medendorp,F,93,1953-10-12 00:00:00,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11
2,2,Eli,Bockman,Male,81,1980-12-16 00:00:00,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16
3,3,Arlin,Dearle,Male,61,1954-01-20 00:00:00,Recruiting Manager,Property,Mass Customer,N,2018-02-01 00:00:00,Yes,15
4,4,Talbot,NaN,Male,33,1961-10-03 00:00:00,NaN,IT,Mass Customer,N,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7
5,5,Sheila-kathryn,Calton,Female,56,1977-05-13 00:00:00,Senior Editor,NaN,Affluent Customer,N,NIL,Yes,8


In [40]:
#checking the details of the dataset
KPMG_CD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 1 to 4000
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   customer_id                          4000 non-null   object
 1   first_name                           4000 non-null   object
 2   last_name                            3875 non-null   object
 3   gender                               4000 non-null   object
 4   past_3_years_bike_related_purchases  4000 non-null   object
 5   DOB                                  3913 non-null   object
 6   job_title                            3494 non-null   object
 7   job_industry_category                3344 non-null   object
 8   wealth_segment                       4000 non-null   object
 9   deceased_indicator                   4000 non-null   object
 10  default                              3698 non-null   object
 11  owns_car                             4000 n

In [41]:
#Checking the number of missing values in each column
KPMG_CD.isnull().sum()

0
customer_id                              0
first_name                               0
last_name                              125
gender                                   0
past_3_years_bike_related_purchases      0
DOB                                     87
job_title                              506
job_industry_category                  656
wealth_segment                           0
deceased_indicator                       0
default                                302
owns_car                                 0
tenure                                  87
dtype: int64

The columns: job_title and job_industry_category have the highest number of missing values however these columns are not sensitive enough to require being dropped from dataset but cannot be replaced by the mode of the column due to the need for accuracy hence each null value will be replaced with "N/A" (Not Available)

In [42]:
#filling the null values in both columns
KPMG_CD['job_title'].fillna('N/A', inplace = True)
KPMG_CD['job_industry_category'].fillna('N/A', inplace = True)

In [43]:
#Recasting in the appropriate datatypes
KPMG_CD['past_3_years_bike_related_purchases'] = KPMG_CD['past_3_years_bike_related_purchases'].astype('float')
KPMG_CD['DOB'] = pd.to_datetime(KPMG_CD['DOB'])
KPMG_CD['tenure'] = KPMG_CD['tenure'].astype('float')
KPMG_CD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 1 to 4000
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   customer_id                          4000 non-null   object        
 1   first_name                           4000 non-null   object        
 2   last_name                            3875 non-null   object        
 3   gender                               4000 non-null   object        
 4   past_3_years_bike_related_purchases  4000 non-null   float64       
 5   DOB                                  3913 non-null   datetime64[ns]
 6   job_title                            4000 non-null   object        
 7   job_industry_category                4000 non-null   object        
 8   wealth_segment                       4000 non-null   object        
 9   deceased_indicator                   4000 non-null   object        
 10  default     

In [44]:
len(KPMG_CD[KPMG_CD.duplicated()])

0

No duplicates were found in the dataset

## checking for accuracy in each column

In [45]:
#checking for accuracy in the gender column
KPMG_CD['gender'].value_counts()

Female    2037
Male      1872
U           88
F            1
Femal        1
M            1
Name: gender, dtype: int64

In [46]:
pd.set_option('mode.chained_assignment', None)

The gender column has other categories than male and female.

In [47]:
#Replacing the 'F' with 'Female'
KPMG_CD['gender'][KPMG_CD['gender'] == 'F']
KPMG_CD['gender'].iloc[0] = KPMG_CD['gender'].iloc[0].replace('F', 'Female')

In [48]:
#Replacing the 'Femal' with 'Female'
KPMG_CD['gender'][KPMG_CD['gender'] == 'Femal']
KPMG_CD['gender'].iloc[53] = KPMG_CD['gender'].iloc[53].replace('Femal', 'Female')

In [49]:
#Replacing the 'M' with 'Male'
KPMG_CD['gender'][KPMG_CD['gender'] == 'M']
KPMG_CD['gender'].iloc[56] = KPMG_CD['gender'].iloc[56].replace('M', 'Male')

In [50]:
#Checking the gender column again
KPMG_CD['gender'].value_counts()

Female    2039
Male      1873
U           88
Name: gender, dtype: int64

In [51]:
#Checking for accuracy in the default column
KPMG_CD['default'].value_counts()

100                                                                                                                                                                                                                   113
1                                                                                                                                                                                                                     112
-1                                                                                                                                                                                                                    111
-100                                                                                                                                                                                                                   99
Ù¡Ù¢Ù£                                                                                                                          

The default column has values that are very incomprehensible, therefore posing high inaccuracy issues.
This column will be dropped from the dataset

In [52]:
KPMG_CD.drop('default', axis = 'columns', inplace = True)
KPMG_CD.dropna(inplace = True)
KPMG_CD.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3792 entries, 1 to 4000
Data columns (total 12 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   customer_id                          3792 non-null   object        
 1   first_name                           3792 non-null   object        
 2   last_name                            3792 non-null   object        
 3   gender                               3792 non-null   object        
 4   past_3_years_bike_related_purchases  3792 non-null   float64       
 5   DOB                                  3792 non-null   datetime64[ns]
 6   job_title                            3792 non-null   object        
 7   job_industry_category                3792 non-null   object        
 8   wealth_segment                       3792 non-null   object        
 9   deceased_indicator                   3792 non-null   object        
 10  owns_car    

Major data quality issues in the Customer Demographic dataset:

1. Missing Values: The dataset had 506  (12.65%) missing values in job_title and 656 (16.4%) missing values in job_industry_category columns. It also had 125 (3.125%) customers whose last names were not documented and another 87 (2.175%) had both missing date of births and tenure values.

2. Accuracy of columns: The columns gender and default had accuracy issues. In gender column, other categories outside of 'female' and 'male' were documented. These categories were: 'U', 'F', 'Femal', and 'M'. It can be assumed that the last three categories are input errors. However more clarity is needed on what gender 'U' is. In the default column, the values are completely incomprehensible.

Mitigation:  Replaced the missing values in job_title and  job_industry_category  with 'N/A' (Not Available) to reduce the number of missing values from 1370 to 415. Replaced misspelt categories ensure consistency.

Recommendation: Enforce a drop-down list for the user entering the data rather than a free text field to increase column accuracy. 

# Checking for quality in the Customer Address dataset

In [53]:
#viewing the dataset
KPMG_CA.head()

,customer_id,address,postcode,state,country,property_valuation
1,1,060 Morning Avenue,2016,New South Wales,Australia,10
2,2,6 Meadow Vale Court,2153,New South Wales,Australia,10
3,4,0 Holy Cross Court,4211,QLD,Australia,9
4,5,17979 Del Mar Point,2448,New South Wales,Australia,4
5,6,9 Oakridge Court,3216,VIC,Australia,9


In [54]:
#checking the details of the dataset
KPMG_CA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 1 to 3999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         3999 non-null   object
 1   address             3999 non-null   object
 2   postcode            3999 non-null   object
 3   state               3999 non-null   object
 4   country             3999 non-null   object
 5   property_valuation  3999 non-null   object
dtypes: object(6)
memory usage: 93.8+ KB


The dataset has no missing values

In [55]:
#recasting the columns in the appropriate datatype
KPMG_CA['property_valuation'] = KPMG_CA['property_valuation'].astype('int')
KPMG_CA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 1 to 3999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         3999 non-null   object
 1   address             3999 non-null   object
 2   postcode            3999 non-null   object
 3   state               3999 non-null   object
 4   country             3999 non-null   object
 5   property_valuation  3999 non-null   int32 
dtypes: int32(1), object(5)
memory usage: 93.8+ KB


In [56]:
#checking for duplicates in the dataset
len(KPMG_CA[KPMG_CA.duplicated()])

0

No duplicate was found in the dataset.

## Checking for accuracy in each column

In [57]:
KPMG_CA['state'].value_counts()

NSW                2054
VIC                 939
QLD                 838
New South Wales      86
Victoria             82
Name: state, dtype: int64

In [58]:
KPMG_CA['country'].value_counts()

Australia    3999
Name: country, dtype: int64

The customer address dataset had no data quality issues.

The dataset had no missing values and all columns passed the accuracy check.

## Using sql to check for data quality in transactions, customer demographic and customer address datasets 

Four datasets were provided namely:
1. Transactions
2. New Customer List
3. Customer Demographic
4. Customer Address

The transactions, customer demographic and customer address datasets all contain information of customers that aren't new and therefore all have customer ids.
The remainder of this notebook contains the sql code implemented to check the quality in all three datasets.

In [59]:
#creating and connecting KPMGdata database
database_connection = sql.connect('c:Documents/KPMGdata.db')

In [60]:
#creating a cursor 
c = database_connection.cursor()

In [61]:
# creating the Transactions table in the database
c.execute(
    '''
            CREATE TABLE transactions (
                transaction_id INTEGER,
                product_id INTEGER,
                customer_id INTEGER,
                transaction_date TEXT,
                online_order TEXT,
                order_status TEXT,
                brand TEXT,
                product_line TEXT,
                product_class TEXT,
                product_size TEXT,
                list_price REAL,
                standard_cost REAL,
                product_first_sold_date TEXT,
                PRIMARY KEY (transaction_id),
                FOREIGN KEY (customer_id) REFERENCES customerdemographic (customer_id),
                FOREIGN KEY (customer_id) REFERENCES customeraddress (customer_id)            
                );
     ''')

In [62]:
# creating the customerdemographic table in the database
c.execute(
    '''
    CREATE TABLE customerdemographic (
                customer_id INTEGER,
                first_name TEXT,
                last_name TEXT,
                gender TEXT,
                past_3_years_bike_related_purchases REAL,
                DOB TEXT,
                job_title TEXT,
                job_industry_category TEXT,
                wealth_segment TEXT,
                deceased_indicator TEXT,
                owns_car TEXT,
                tenure INTEGER,
                PRIMARY KEY(customer_id),
                FOREIGN KEY(customer_id) REFERENCES transactions(customer_id),
                FOREIGN KEY(customer_id) REFERENCES customeraddress(customer_id)
                );
    '''
)

In [63]:
#creating the customeraddress table in the database
c.execute(
    '''
            CREATE TABLE customeraddress (
                customer_id INTEGER,
                address TEXT,
                postcode INTEGER,
                state TEXT,
                country TEXT,
                property_valuation INTEGER,
                PRIMARY KEY (customer_id),
                FOREIGN KEY (customer_id) REFERENCES transactions (customer_id),
                FOREIGN KEY (customer_id) REFERENCES customerdemographic (customer_id)            
                );
     ''')

In [64]:
#filling the table with information from the datasets
KPMG_TSC.to_sql('transactions', database_connection, if_exists = 'append', index = False)
KPMG_CD.to_sql('customerdemographic', database_connection, if_exists = 'append', index = False)
KPMG_CA.to_sql('customeraddress', database_connection, if_exists = 'append', index = False)

3999

In [65]:
#reading the first five rows of the transactions table
pd.read_sql('''
            SELECT * 
            FROM transactions 
            LIMIT 5
            ''', database_connection)

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25 00:00:00,0,Approved,Solex,Standard,medium,medium,71.49,53.62,2012-12-02 00:00:00
1,2,3,3120,2017-05-21 00:00:00,1,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,2014-03-03 00:00:00
2,3,37,402,2017-10-16 00:00:00,0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,1999-07-20 00:00:00
3,4,88,3135,2017-08-31 00:00:00,0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,1998-12-16 00:00:00
4,5,78,787,2017-10-01 00:00:00,1,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,2015-08-10 00:00:00


In [66]:
#checking for customers who have performed trasactions but have no customer demographic or customer address column.
pd.read_sql(
    '''
    SELECT DISTINCT t.customer_id, cd.first_name, cd.last_name, ca.address
    FROM transactions AS t
    LEFT JOIN customerdemographic AS cd
    ON t.customer_id = cd.customer_id
    LEFT JOIN customeraddress AS ca
    ON t.customer_id = ca.customer_id
    WHERE ca.address IS NULL OR cd.first_name IS NULL
''', database_connection
)

,customer_id,first_name,last_name,address
0,2426,None,None,4 Becker Center
1,3053,None,None,274 Ludington Trail
2,475,None,None,781 Dorton Avenue
3,857,None,None,18 Parkside Plaza
4,3473,None,None,7279 Steensland Circle
5,2000,None,None,59170 Westridge Junction
6,927,None,None,2309 Gulseth Way
7,211,None,None,78 Rockefeller Park
8,22,Deeanne,Durtnell,None
9,1038,None,None,85 Laurel Trail


Five customers have performed transactions but have no customer address attached to their customer id, however, one of this five customers also has no customer demographic record present.

In [67]:
#Checking for transactions details of customers with no demographic or address record
customer_idTSC = pd.read_sql(
    '''
    SELECT DISTINCT t.customer_id, cd.first_name, cd.last_name, ca.address, t.transaction_date, t.brand, t.standard_cost
    FROM transactions AS t
    LEFT JOIN customerdemographic AS cd
    ON t.customer_id = cd.customer_id
    LEFT JOIN customeraddress AS ca
    ON t.customer_id = ca.customer_id
    WHERE ca.address IS NULL OR cd.first_name IS NULL
''', database_connection
)
customer_idTSC

,customer_id,first_name,last_name,address,transaction_date,brand,standard_cost
0,2426,None,None,4 Becker Center,2017-04-03 00:00:00,Norco Bicycles,818.01
1,3053,None,None,274 Ludington Trail,2017-06-20 00:00:00,Solex,589.27
2,475,None,None,781 Dorton Avenue,2017-05-07 00:00:00,Trek Bicycles,154.40
3,857,None,None,18 Parkside Plaza,2017-11-27 00:00:00,Giant Bicycles,993.66
4,3473,None,None,7279 Steensland Circle,2017-12-18 00:00:00,Giant Bicycles,1759.85
...,...,...,...,...,...,...,...
1064,2073,None,None,9499 Alpine Drive,2017-02-21 00:00:00,WeareA2B,513.85
1065,657,None,None,9912 Eagle Crest Lane,2017-03-01 00:00:00,Giant Bicycles,173.18
1066,3401,None,None,6851 Orin Parkway,2017-10-18 00:00:00,Trek Bicycles,459.71
1067,275,None,None,07158 Roth Pass,2017-07-02 00:00:00,Trek Bicycles,596.55


In [68]:
#saving the four cleaned datasets as excel sheets in the same workbook
writer = ExcelWriter('c:Documents/KPMG clean data.xlsx')
KPMG_TSC.to_excel (writer,'Transactions', index = False)
KPMG_NCL.to_excel (writer,'NewCustomerList', index = False)
KPMG_CD.to_excel (writer, 'CustomerDemographic', index = False)
KPMG_CA.to_excel (writer, 'CustomerAddress', index = False)
writer.save()

In [69]:
#saving the dataset with customer id outliers
customer_idTSC.to_excel('c:Documents/customer_idTSC.xlsx', index = False)